In [8]:
import pyspark.sql.functions as F
import pandas as pd
from pyspark.sql.types import *
from notebookutils import mssparkutils as mu
from pyspark.sql import SparkSession

StatementMeta(, a8f03415-f37f-4ad9-8898-1ab09922652d, 10, Finished, Available, Finished)

In [9]:
spark = SparkSession.builder.getOrCreate()

try:
    spark.sql("CREATE SCHEMA if NOT exists gold")
    print("Schema created Bronze")

except Exception as ex:
    print("error in schemas")

StatementMeta(, a8f03415-f37f-4ad9-8898-1ab09922652d, 11, Finished, Available, Finished)

Schema created Bronze


In [21]:
df_dim_projects = (
    spark.table("silver.projects")
    .select(
        F.col("ProjectID"),
        F.col("ProjectName"),
        F.col("City"),
        F.col("Region"),
        F.col("LaunchYear"),
        F.col("Status")
    )
)

df_dim_properties =(
    spark.table("silver.properties")
    .select(
        F.col("PropertyID"),
        F.col("ProjectID"),
        F.col("PropertyType"),
        F.col("Size_m2"),
        F.col("Bedrooms"),
        F.col("Bathrooms"),
        F.col("AvailabilityStatus")
    ).distinct()
)


df_dim_brokers = (
    spark.table("silver.brokers")
    .select(
        F.col("BrokerID"),
        F.col("BrokerName"),
        F.col("Region"),
        F.col("Email")

    )
    .distinct()
)


df_dim_clients = (
    spark.table("silver.clients")
    .select(
        F.col("ClientID"),
        F.col("FirstName"),
        F.col("LastName"),
        F.col("Email"),
        F.col("Region")
    )
    .distinct()
)

df_dim_campaigns = (
    spark.table("silver.campaigns")
    .select(
        F.col("CampaignID"),
        F.col("CampaignName"),
        F.col("Channel"),
        F.col("Budget"),
        F.col("StartDate"),
        F.col("EndDate"),
        F.col("DaysBetweenCampaign")

    )
    .distinct()
)

df_fact_leads = (
    spark.table("silver.leads")
    .select(
        F.col("LeadID"),
        F.col("ClientID"),
        F.col("PropertyID"),
        F.col("CampaignID"),
        F.col("LeadDate"),
        F.col("LeadSource")
    )
)

df_fact_sales = (
    spark.table("silver.sales")
    .select(
        F.col("SaleID"),
        F.col("ClientID"),
        F.col("BrokerID"),
        F.col("SaleDate"),
        F.col("SalePriceUSD")
    )
)







StatementMeta(, a8f03415-f37f-4ad9-8898-1ab09922652d, 23, Finished, Available, Finished)

In [22]:
df_dim_projects.write.format("delta").mode("overwrite").saveAsTable("gold.dim_projects")
df_dim_properties.write.format("delta").mode("overwrite").saveAsTable("gold.dim_properties")
df_dim_brokers.write.format("delta").mode("overwrite").saveAsTable("gold.dim_brokers")
df_dim_clients.write.format("delta").mode("overwrite").saveAsTable("gold.dim_clients")
df_dim_campaigns.write.format("delta").mode("overwrite").saveAsTable("gold.dim_campaigns")
df_fact_leads.write.format("delta").mode("overwrite").saveAsTable("gold.fact_leads")
df_fact_sales.write.format("delta").mode("overwrite").saveAsTable("gold.fact_sales")


StatementMeta(, a8f03415-f37f-4ad9-8898-1ab09922652d, 24, Finished, Available, Finished)